### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Esophageal_Cancer/GSE112264'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Based on the background info, it appears the dataset is about miRNA, not gene expression
is_gene_available = False

# Check for the presence of key variables in sample characteristics
for key, values in clinical_data.items():
    if any("disease state: Esophageal Cancer" in val for val in values):
        trait_row = key
    elif any("age:" in val for val in values):
        age_row = key
    elif any("Sex:" in val for val in values):
        gender_row = key

# Ensure all keys are set and valid
trait_row = trait_row if trait_row and len(set(clinical_data.get(trait_row, []))) > 1 else None
age_row = age_row if age_row and len(set(clinical_data.get(age_row, []))) > 1 else None
gender_row = gender_row if gender_row and len(set(clinical_data.get(gender_row, []))) > 1 else None

# Define conversion functions
def convert_trait(value):
    # Extract value after colon and check if it matches 'Esophageal Cancer'
    extracted_value = value.split(':')[-1].strip()
    return 1 if extracted_value == "Esophageal Cancer" else 0

def convert_age(value):
    # Extract value after colon and convert to int
    try:
        return int(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    # Extract value after colon and convert to binary
    extracted_value = value.split(':')[-1].strip()
    return 1 if extracted_value == "Male" else 0

# Save cohort info
save_cohort_info('GSE112264', './preprocessed/Esophageal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is not None, extract clinical data
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Esophageal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Esophageal_Cancer/trait_data/GSE112264.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
